
우리가 알거니와 하나님을 사랑하는 자 곧 그의 뜻대로 부르심을 입은 자들에게는 모든 것이 합력하여 선을 이루느니라. 롬8:28  

And we know that in all things God works for the good of those who love him, who have been called according to his purpose. (Rom 8:28)

-------

# Welcome to "AI for All"

Lecture Notes by idebtor@gmail.com, Handong Global University

# 제 1 강 신경망을 내 손으로 만져보기(Gate)

---------



## 1. AND 연산 

AND 연산은 모두 입력이 참일 때, 참을 결과로 출력하는 연산입니다.  다음의 진리표에 표시된 바와 같습니다. 

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/truthtable.png?raw=true" width="400">
<center>그림 1:  OR, AND, NAND, XOR 진리표 </center>


이를 넘파이 배열로 나타내면 다음과 같습니다.  

In [1]:
import numpy as np
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 
y = np.array([[0], [0], [0], [1]])
print(x)
print(y)

[[0 0]
 [0 1]
 [1 0]
 [1 1]]
[[0]
 [0]
 [0]
 [1]]


기계 학습의 관점으로 AND 로직 연산을 바라보면, 입력은 두 개 특성($x_1, x_2$)을 가지고 있으며, 네 개의 샘플이 있으며, 따라서 네 개의 각 샘플에 대한 레이블(혹은 타깃)이 있습니다. 이러한 샘플들과 레이블이 곧 퍼셉트론의 훈련 데이터셋이 됩니다. 

우리는 사실상 위에서 훈련 데이터셋을 준비한 것입니다. 테스트 혹은 예측을 위한 데이터셋은 존재하지 않으므로, 입력 데이터셋을 그대로 테스트를 위해서도 사용할 것입니다. 

다음은 데이터셋의 샘플 갯수와 레이블 갯수가 같은지 점검하고, 샘플 갯수와 에폭을 설정합니다. 

In [2]:
assert len(x) == len(y)
samples = len(x)
epochs = 1000

모델을 학습시키기 전에, 가중치와 편향도 정규 분포를 가지는 난수로 초기화 해줍니다. 또한, 학습률(eta)은 0.1로 설정하였습니다.  입력값(특성)이 두개이므로, 가중치도 각각에 맞춰서 두개를 선언해줍니다.

In [3]:
import numpy as np                 # import tensorflow as tf 
w = np.array([1.0, 1.0])            # w = tf.random.normal([2], 0, 1)         # mean = 0, std dev = 1
b = np.array([1.0])                 # b = tf.random.normal([1], 0, 1) 
eta = 0.1                           # 학습률(learning rate)

이제 학습을 시켜보겠습니다. 앞에서 만든 편향을 가진 뉴런처럼 코드를 구성하면 됩니다. 각각 네가지 경우를 한번씩 학습할 때마다, 네가지 를 예측한 값과 실제 값의 차이로 계산할 수 있는 SSE(sum of squared error)를 구해서 살펴보겠습니다.

In [4]:
def activation(x):  
    return 1/(1 + np.exp(-x))         # sigmoid function

In [5]:
for i in range(epochs): 
    sse_i = 0 
    for j in range(samples): 
        z = np.sum(x[j] * w) + b        # net input
        a = activation(z)               # apply activation function
        error = y[j][0] - a             # error
        w = w + eta * error * x[j]      # new weight
        b = b + eta * error             # new bias
        sse_i += error * error
    if (i + 1) % 100 == 0: 
        print(i + 1, sse_i / 100)

100 [0.0023528]
200 [0.00116179]
300 [0.00068577]
400 [0.00044865]
500 [0.00031457]
600 [0.0002319]
700 [0.00017757]
800 [0.00014008]
900 [0.00011317]
1000 [9.32504265e-05]


이러한 학습을 통해서 결정된 것은 무엇입니까? 

모델의 가중치와 편향입니다.  

이제 학습을 통해서 얻은 기계 학습 모델에 새로운 데이터를 제공하여 출력을 예측하고자 합니다. 다만, 여기서는 예측에 사용할 데이터가 없으므로, 학습에 이미 사용한 데이터를 사용하여 각각의 케이스를 예측하는 것이 합당합니다. 이를 계산하는 방법은 다음과 같이 퍼셉트론의 __순입력(z)__를 구하고 이를 __활성화 함수에 적용하면__ 예측값을 계산할 수 있습니다. __즉 모델의 순뱡향(forpass)만 계산하면 예측값을 산출할 수 있습니다.__ 이러한 두 연산을 합하여 forpass() 메소드로 구현합니다. 
```
z = np.sum(x[i] * w) + b 
a = activation(z)
```

In [6]:
for i in range(samples): 
    print('{} {} y_hat:{}'.format(x[i], y[i], activation(np.sum(x[i] * w) + b)))

[0 0] [0] y_hat:[0.00018617]
[0 1] [0] y_hat:[0.04842622]
[1 0] [0] y_hat:[0.04870589]
[1 1] [1] y_hat:[0.93330356]


위와 같이, 실제로 1이 나와야하는 값은 1에 가깝게, 0에 가까워야하는 값은 0에 가깝게 예측이 되었습니다. `[0 0]` 케이스의 경우에는 다른 케이스들 보다 더 0에 가까운걸로 보아 더 확실한 케이스라는 점을 알 수 있습니다.

## 2. OR 연산

OR 연산은 입력들 중에 하나라도 참이면, 참을 결과로 출력하는 연산입니다.  이를 넘파이 배열로 나타내면 다음과 같습니다. 

In [7]:
x = np.array([[1, 1], [1, 0], [0, 1], [0, 0]], "float32") 
y = np.array([[1], [1], [1], [0]],  "float32")

w = np.array([1.0, 1.0])            # w = tf.random.normal([2], 0, 1)         # mean = 0, std dev = 1
b = np.array([1.0])                 # b = tf.random.normal([1], 0, 1) 
eta = 0.1                           # 학습률(learning rate)

이를 AND와 같은 가중치, 편향, 학습률 및 학습 모델로 학습을 시키면 다음과 같습니다. 

In [8]:
for i in range(epochs): 
    sse_i = 0 
    for j in range(samples): 
        z = np.sum(x[j] * w) + b        # net input
        a = activation(z)               # apply activation function
        error = y[j][0] - a             # error
        w = w + eta * error * x[j]      # new weight
        b = b + eta * error             # new bias
        sse_i += error * error
    if (i + 1) % 100 == 0: 
        print(i + 1, sse_i / 100)

100 [0.00141742]
200 [0.00059027]
300 [0.0003127]
400 [0.00019075]
500 [0.00012751]
600 [9.08519009e-05]
700 [6.78295892e-05]
800 [5.24802686e-05]
900 [4.1759897e-05]
1000 [3.39897801e-05]


학습을 진행할 수록 error값의 합이 0에 가까워지는것을 확인할 수 있습니다. 각각의 케이스들을 해당 가중치와 편향으로 예측해보겠습니다.

In [9]:
for i in range(samples): 
    print('{} {} y_hat:{}'.format(x[i], y[i], activation(np.sum(x[i] * w) + b)))

[1. 1.] [1.] y_hat:[0.99997728]
[1. 0.] [1.] y_hat:[0.97979667]
[0. 1.] [1.] y_hat:[0.97972554]
[0. 0.] [0.] y_hat:[0.05054725]


## 3. XOR 연산
XOR은 AND나 OR연산과는 다르게 홀수 개의 입력이 참일 때, 결과가 참이 됩니다. 입력을 두개라고 한다면, 위의 진리표에 나타난 바와 같습니다.

이를 넘파이 배열로 나타내면 다음과 같습니다. 

In [10]:
x = np.array([[1, 1], [1, 0], [0, 1], [0, 0]],  "float32") 
y = np.array([[0], [1], [1], [0]],  "float32")

w = np.array([1.0, 1.0])            # w = tf.random.normal([2], 0, 1)         # mean = 0, std dev = 1
b = np.array([1.0])                 # b = tf.random.normal([1], 0, 1) 
eta = 0.1                           # 학습률(learning rate)

이를 AND와 같은 가중치, 편향, 학습률 및 학습 모델로 학습을 시키면 다음과 같습니다. 

In [11]:
for i in range(epochs): 
    sse_i = 0 
    for j in range(samples): 
        z = np.sum(x[j] * w) + b        # net input
        a = activation(z)               # apply activation function
        error = y[j][0] - a             # error
        w = w + eta * error * x[j]      # new weight
        b = b + eta * error             # new bias
        sse_i += error * error
    if (i + 1) % 100 == 0: 
        print(i + 1, sse_i / 100)

100 [0.01053073]
200 [0.01052111]
300 [0.01051963]
400 [0.01051935]
500 [0.01051929]
600 [0.01051928]
700 [0.01051928]
800 [0.01051928]
900 [0.01051928]
1000 [0.01051928]


위와 같이, 특정 학습 지점부터 에러 값에 변화가 거의 없습니다. 이것이 과연 학습이 잘된걸까요? 계산된 가중치와 편향을 가지고 각 케이스를 계산해보았습니다.

In [12]:
for i in range(samples): 
    print('{} {} y_hat:{}'.format(x[i], y[i], activation(np.sum(x[i] * w) + b)))

[1. 1.] [0.] y_hat:[0.51281764]
[1. 0.] [1.] y_hat:[0.51281763]
[0. 1.] [1.] y_hat:[0.5]
[0. 0.] [0.] y_hat:[0.49999999]


위와 같이, 결과가 0 혹은 1에 가까운 값이 나와 하지만, 모든 케이스가 0.5에 가깝게 나오고 있습니다. 이것은 원하던 결과가 아닙니다. 결과를 해석해보자면, 가중치와 편향 값은 모두 케이스 순서에 의존적이 된다는 것을 알 수 있습니다. 먼저 들어간 `[1 1]`이라는 케이스가 네번째에 들어가는 `[0 0]`이라는 케이스보다 영향을 준다는 것입니다. `[1 1]`이라는 케이스가 먼저 들어가서 가중치와 편향에 중대한 영향을 미치고 이 값들을 가지고 학습을 진행한다는 것이 문제입니다.

그러면 XOR문제는 풀지 못하는 걸까요? 여러 층의 퍼셉트론을 사용하면 해결이 됩니다.

## 4. 단층 퍼셉트론에서의 XOR 문제점

인공 신경망에서는 단층 퍼셉트론으로 XOR 연산이 불가능하다는 것은 마빈 민스키 등에 의해서 밝혀졌습니다. 이러한 내용이 밝혀지면서 인공지능의 겨울이 찾아왔었습니다. 그야말로 전설같은 이야기로 잘 알려져 있습니다. 

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/ai4all-history.jpg?raw=true" width="900">
<center>그림 2: XOR 문제와 인공 지능의 발전사</center>

인공 신경망(ANN, Aritificial Neural Networks)은 1943년 신경생리학자 Warren McCulloch과 수학자 Walter Pitts가 'A Logical Calculus of Ideas Immanent In Nervous Activity' 처음 소개했습니다. 그 이후 1960년대까지 당싱에 등장한 인공 신경망을 통해 사람들은 지능을 가지 기계가 상당히 엄청난 일을 해낼 것이라 생각했습니다. 그러나, 위의 그림(출처: [beamandrew's blog](https://beamandrew.github.io/deeplearning/2017/02/23/deep_learning_101_part1.html))처럼 사람들의 기대와는 달리 인공 신경망으로 XOR문제를 해결할 수 없게 되었고, 인공 지능과 관련한 연구는 암흑기로 접어 들게 되었다. 그래도, 1990년 대에는 SVM과 성능이 좋은 다른 머신러닝 알고리즘들이 나올 정도도 꾸준한 연구가 진행은 되고 있었습니다. 

2000년대에 들어서면서 인공 신경망은 2012년 ILSVRC2012 대회에서 인공 신경망을 깊게 쌓은 딥러닝 모델인 AlexNet이 압도적인 성적으로 우승하면서 다시금 주목받게 되었습니다. 이렇게 인공 신경망에 기반을 둔 딥러닝이 다시 주목받게 된 계기가 되었습니다. 

인공 지능의 발전사에서 약간의 의미가 있었던 XOR 문제를 아래 그림과 같은 다층 신경망을 통해 이제 어렵지 않게 풀어볼 수 있는 문제가 되었습니다. 

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/ai4all-xor.png?raw=true" width="600">
<center>그림 3:  XOR 연산을 위한 다층 인공 신경망</center>

# Final Exam begins here

## Exercises 

#### AND, NAND, OR, NOR 게이트 로직을 다룰 수 있는 Gate라는 클래스를 만들고 출력을 확인하십시오. 

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/truthtable.png?raw=true" width="400">
<center>그림 1:  OR, AND, NAND, XOR 진리표 </center>

#### Part 1:  Gate 클래스 정의하기 (sse 인스턴스 변수와 코드 포함)

#### Solution:

In [13]:
import numpy as np

class Gate:
    def __init__(self):
        self.w = np.array([1.0, 1.0])         
        self.b = np.array([1.0]) 
        self.eta = 0.1

    def forpass(self, x):
        z = np.sum(x * self.w) + self.b
        a = self.activation(z)
        #print('forpass x{} z{} a{}'.format(x, z, a))
        return a

    def backprop(self, x, error): 
        w_grad = x * error
        b_grad = 1 * error
        return w_grad, b_grad
    
    def activation(self, z): 
        a = 1/(1 + np.exp(-z))
        return a

    def fit(self, x, y, epochs = 1000):         # default epochs = 1000
        self.sse = np.zeros(epochs)             # initialize sse array as 0
        
        for i in range(epochs):                  # epoch만큼 반복합니다. 
            sse_i = 0                            # compute sse per epoch 
            
            for x_i, y_i in zip(x, y):          # 모든 샘플에 대해 반복합니다.
                a =                              # 순전파 계산
                error =                          # 오차 계산
                w_grad, b_grad =                 # 역방향 계산
                self.w +=                        # 가중치 조정
                self.b +=                        # 편향 조정
                sse_i +=                         # accumulate error for each sample
            self.sse[i] =                        # save each sse for every epoch in the object

#### 입력 x와 y_and, y_nand, y_or, y_xor 레이블(타깃)을 정의하기 

In [14]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 
y_and  = np.array([[0], [0], [0], [1]])
y_nand = np.array([[1], [1], [1], [0]])
y_or   = np.array([[0], [1], [1], [1]])
y_nor  = np.array([[1], [0], [0], [0]])

#### Part 2: gate_and 객체와 훈련하기  (epochs = 500)

- gate_and 객체를 생성하기
- 훈련하기
- mse 구하기 
- 모델의 결과와 예측을 출력하기 

```
AND Gate
Weight: [4.31247748 4.30140545]
  Bias: [-6.6310874]
[0 0] [0] y_hat:[0.00131699]
[0 1] [0] y_hat:[0.08869437]
[1 0] [0] y_hat:[0.08959338]
[1 1] [1] y_hat:[0.87897885]
```
#### Solution:

In [15]:
## your code here


print("AND Gate")
print("Weight:", gate_and.w)
print("  Bias:", gate_and.b)

for i in range(len(x)): 
    print('your code here')

AND Gate
Weight: [6.98823409 6.9851074 ]
  Bias: [-10.64790991]
[0 0] [0] y_hat:[2.37498662e-05]
[0 1] [0] y_hat:[0.02501851]
[1 0] [0] y_hat:[0.02509489]
[1 1] [1] y_hat:[0.96529103]


#### Part 3: epoch에 따른 mse 시각화 하기

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/ai4all-gate-mse.png?raw=true" width="400">
<center>그림 4: Gradient Descent MSE for Logic Gate</center>

#### Solution:

In [ ]:
import matplotlib.pyplot as plt

print('your code here')

plt.xlabel('epoch')
plt.ylabel('mse')
plt.title('Gradient Descent MSE for Logic Gate')
plt.show()

#### Part 4: NAND Gate

- gate_nand 객체를 생성하기
- 훈련하기
- mse 구하기 
- 모델의 결과와 예측을 출력하기 

```
NAND Gate
Weight: [-4.25914936 -4.24781832]
  Bias: [6.55111244]
[0 0] [1] y_hat:[0.99857351]
[0 1] [1] y_hat:[0.90914949]
[1 0] [1] y_hat:[0.90820923]
[1 1] [0] y_hat:[0.12391631]
```
#### Solution:

In [25]:
print('your code here')

print("NAND Gate")
print("Weight:", gate_nand.w)
print("  Bias:", gate_nand.b)

for i in range(len(x)): 
    print('your code here')

NAND Gate
Weight: [-6.97354402 -6.9703949 ]
  Bias: [10.625858]
[0 0] [1] y_hat:[0.99997572]
[0 1] [1] y_hat:[0.97480184]
[1 0] [1] y_hat:[0.97472437]
[1 1] [0] y_hat:[0.03495609]


#### Part 4: OR Gate

- gate_or 객체를 생성하기
- 훈련하기
- mse 오차 구하기 
- 모델의 결과와 예측을 출력하기 

```
OR Gate
Weight: [5.42570445 5.43193129]
  Bias: [-2.21461759]
[0 0] [0] y_hat:[0.09844548]
[0 1] [1] y_hat:[0.96148065]
[1 0] [1] y_hat:[0.96124937]
[1 1] [1] y_hat:[0.99982368]
```
#### Solution:

In [26]:
print('your code here')

print("OR Gate")
print("Weight:", gate_or.w)
print("  Bias:", gate_or.b)
for i in range(len(x)): 
    print('your code here')

OR Gate
Weight: [8.22173119 8.22364178]
  Bias: [-3.64329517]
[0 0] [0] y_hat:[0.02549878]
[0 1] [1] y_hat:[0.98985268]
[1 0] [1] y_hat:[0.98983347]
[1 1] [1] y_hat:[0.99999724]


#### Part 4: NOR gate

- gate_nor 객체를 생성하기
- 훈련하기
- mse 구하기 
- 모델의 결과와 예측을 출력하기 

```
NOR Gate
Weight: [-5.33364922 -5.34053195]
  Bias: [2.16668323]
[0 0] [1] y_hat:[0.8972175]
[0 1] [0] y_hat:[0.04016179]
[1 0] [0] y_hat:[0.04042795]
[1 1] [0] y_hat:[0.00020191]
```
#### Solution:

In [27]:
print('your code here')

print("NOR Gate")
print("Weight:", gate_nor.w)
print("  Bias:", gate_nor.b)
# print("   MSE:", mse)
for i in range(len(x)): 
    print('your code here')

NOR Gate
Weight: [-6.76459101 -6.76836513]
  Bias: [2.9043104]
[0 0] [1] y_hat:[0.9480591]
[0 1] [0] y_hat:[0.02055152]
[1 0] [0] y_hat:[0.02062763]
[1 1] [0] y_hat:[2.42118118e-05]


## XOR Logic Gate 만들기

우리는 위에서 Gate 클래스를 사용하여, gate_and, gate_or, gate_nand, gate_nor과 다양한 논리회로 기능을 할 수 있는 객체를 만들어 낼 수 있었지만, 그러나 gate_xor 논리회로를 만들 수 없다는 것을 알게 되었습니다. 그런데, 이러한 논리회로들을 여러 개(layer)으로 조합하면, 아래 그림들과 같이 XOR 논리 회로를 만들 수 있습니다. 

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/xor.png?raw=true" width="600">
<center>그림 5:  XOR 연산자 만들기</center>

이를 활용하여 gate_xor() 함수를 두 가지 방법으로 만들어 보십시오. gate_xor 객체 혹은 클래스를 만드는 것은 아니라는 것을 유의하십시오. 

#### Step 1: gate_nand, gate_or, gate_and 객체 테스트 하기

여기서는 gate_xor()에서 사용할 gate_nand, gate_or, gate_and 객체들이 제대로 작동하는지 확인하십시오. 다음은 각 객체들의 결과 값들의 한 예입니다. 구체적인 값들을 다를 수 있지만, 1 혹은 0에 가까운 값으로 산출되어야 합니다.

```
NAND [0 0] [0.99997572]
NAND [0 1] [0.97480184]
NAND [1 0] [0.97472437]
NAND [1 1] [0.03495609]
  OR [0 0] [0.02549878]
  OR [0 1] [0.98985268]
  OR [1 0] [0.98983347]
  OR [1 1] [0.99999724]
 AND [0 0] [2.37498662e-05]
 AND [0 1] [0.02501851]
 AND [1 0] [0.02509489]
 AND [1 1] [0.96529103]
 ```

In [28]:
# gate_xor()에서 사용할 gate_nand, gate_or, gate_and 객체 기능 확인하기
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 

for i in range(len(x)): 
    yhat = gate_nand.forpass(x[i])
    print("NAND", x[i], yhat)

for i in range(len(x)): 
    yhat = gate_or.forpass(x[i])
    print("  OR", x[i], yhat)

for i in range(len(x)): 
    yhat = gate_and.forpass(x[i])
    print(" AND", x[i], yhat)

NAND [0 0] [0.99997572]
NAND [0 1] [0.97480184]
NAND [1 0] [0.97472437]
NAND [1 1] [0.03495609]
  OR [0 0] [0.02549878]
  OR [0 1] [0.98985268]
  OR [1 0] [0.98983347]
  OR [1 1] [0.99999724]
 AND [0 0] [2.37498662e-05]
 AND [0 1] [0.02501851]
 AND [1 0] [0.02509489]
 AND [1 1] [0.96529103]


#### Step 2: gate_xor() 함수 구현하기 

위의 그림에 있는 논리회로를 활용하여 gate_xor()함수를 구현하고 테스트해서 확인하십시오. 
```
 XOR [0 0] [0.02983645]
 XOR [0 1] [0.95599548]
 XOR [1 0] [0.95596705]
 XOR [1 1] [0.03172103]
```

참고로, 아래는 gate_xor()함수 안에서 디버깅을 위해 출력한 입력 x, 중간값 h1, h2, h12 및 출력 yhat입니다.  

```
 XOR [0 0] [0.99997572] [0.02549878] [0.99997572 0.02549878] [0.02983645]
 XOR [0 1] [0.97480184] [0.98985268] [0.97480184 0.98985268] [0.95599548]
 XOR [1 0] [0.97472437] [0.98983347] [0.97472437 0.98983347] [0.95596705]
 XOR [1 1] [0.03495609] [0.99999724] [0.03495609 0.99999724] [0.03172103]
```

Note: forpass에 입력하는 값의 형상(shape) 혹은 차원(dim)에 유의하십시오. forpass()가 반환하는 값, 예를 들면, h1은 scalar가 아니라 ndarray type(형식)이고, forpass() 메소드에 넘겨주어야 하는 값의 type은 `x[1] = [0, 1]`와 같이 1차원의 1x2의 형상(shape)인 ndarray type입니다. 이를 위해 np.array()를 사용해 보는 것도 좋습니다. 

In [29]:
# gate_xor() 함수 구현하기 
# use gate_nand, gate_and and gate_or 

def gate_xor(x):
    
    print('your code here')
    
    #print(" XOR", x, h1, h2, h12, yhat)     # for debugging 
    return yhat

In [30]:
# gate_xor() 함수 테스트하기
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 

for i in range(len(x)): 
    yhat = gate_xor(x[i])
    print(" XOR", x[i], yhat)

 XOR [0 0] [0.02983645]
 XOR [0 1] [0.95599548]
 XOR [1 0] [0.95596705]
 XOR [1 1] [0.03172103]


#### Step 3: gate_xor() 함수 구현하기 

아래 그림에 있는 논리회로를 완성해 보십시오. 이를 활용하여 gate_xor()함수를 구현하고 테스트해서 확인하십시오. 

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/xor_nand_truthtable.png?raw=true" width="400">
<center>그림 6: NAND로 구성된 XOR진리표 </center>

아래 그림에서 각 Gate 앞에 있는 annotation을 변수 이름으로 사용하면 편리합니다. 

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/xor_nand_logic.png?raw=true" width="400">
<center>그림 7: NAND로 구성된 XOR논리 회로 </center>

테스트 결과는 다음과 같습니다. 여러 분의 값들은 다를 수 있지만, 0 혹은 1에 다음과 같은 형태로 수렴해야 합니다. 

```
 XOR [0 0] [0.02983645]
 XOR [0 1] [0.95599548]
 XOR [1 0] [0.95596705]
 XOR [1 1] [0.03172103]
```

참고로, 아래는 gate_xor()함수 안에서 디버깅을 위해 출력한 입력 x, 중간값 h1, x1h1, h2, h3 및 출력 yhat입니다.  

```
 XOR [0 0] [0.99997572] [0.         0.99997572] [0.97480599] [0.97472854] [0.04897468]
 XOR [0 1] [0.97480184] [0.         0.97480184] [0.97877463] [0.04139336] [0.97102171]
 XOR [1 0] [0.97472437] [1.         0.97472437] [0.04141164] [0.97872202] [0.97111137]
 XOR [1 1] [0.03495609] [1.         0.03495609] [0.96797397] [0.96806805] [0.05354669]
```

Note: forpass에 입력하는 값의 형상(shape) 혹은 차원(dim)에 유의하십시오. forpass()가 반환하는 값, 예를 들면, h1은 scalar가 아니라 ndarray type(형식)이고, forpass() 메소드에 넘겨주어야 하는 값의 type은 `x[1] = [0, 1]`와 같이 1차원의 1x2의 형상(shape)인 ndarray type입니다. 이를 위해 np.array()를 사용해 보는 것도 좋습니다. 

In [21]:
# gate_xor() 함수 구현하기 
# use gate_nand only

def gate_xor(x):
    
    print('your code here')
    
    #print(" XOR", x, h1, x1h1, h2, h3, yhat)     # for debugging 
    return yhat

In [31]:
# gate_xor() 함수 테스트하기
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 

for i in range(len(x)): 
    yhat = gate_xor(x[i])
    print(" XOR", x[i], yhat)

 XOR [0 0] [0.02983645]
 XOR [0 1] [0.95599548]
 XOR [1 0] [0.95596705]
 XOR [1 1] [0.03172103]


#### Step 4: yet another yetano_xor() 함수 구현하기 

아래 그림에 있는 논리회로를 완성해 보십시오. 이를 활용하여 gate_xor()함수를 구현하고 테스트해서 확인하십시오. 

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/xor_nor_truthtable.png?raw=true" width="400">
<center>그림 8: NOR로 구성된 XOR진리표 </center>

아래 그림에서 각 Gate 앞에 있는 annotation을 변수 이름으로 사용하면 편리합니다.

<img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/xor_nor_logic.png?raw=true" width="400">
<center>그림 8: NOR로 구성된 XOR진리표 </center>

테스트 결과는 다음과 같습니다. 여러 분의 값들은 다를 수 있지만, 0 혹은 1에 다음과 같은 형태로 수렴해야 합니다. 

```
 XOR [0 0] [0.02894936]
 XOR [0 1] [0.92881756]
 XOR [1 0] [0.92882857]
 XOR [1 1] [0.0290579]
```

참고로, 아래는 yetano_xor()함수 안에서 디버깅을 위해 출력한 입력 x, 중간값 x11, x12, h12, h43 및 출력 yhat입니다.  

```
 XOR [0 0] [0 0] [0 0] [0.9480591] [0.9480591 0.9480591] [4.88976716e-05 9.48059104e-01] [0.02894936]
 XOR [0 1] [0 0] [0 1] [2.42118118e-05] [9.48059104e-01 2.42118118e-05] [0.02905482 0.02055152] [0.92881756]
 XOR [1 0] [1 1] [1 0] [0.9480591] [2.42118118e-05 9.48059104e-01] [0.02895405 0.02062763] [0.92882857]
 XOR [1 1] [1 1] [1 1] [2.42118118e-05] [2.42118118e-05 2.42118118e-05] [9.48042967e-01 2.42118118e-05] [0.0290579]
```

Note: forpass에 입력하는 값의 형상(shape) 혹은 차원(dim)에 유의하십시오. forpass()가 반환하는 값, 예를 들면, h1은 scalar가 아니라 ndarray type(형식)이고, forpass() 메소드에 넘겨주어야 하는 값의 type은 `x[1] = [0, 1]`와 같이 1차원의 1x2의 형상(shape)인 ndarray type입니다. 이를 위해 np.array()를 사용해 보는 것도 좋습니다. 

In [38]:
# yet another xor() 함수 구현하기 
# use gate_nor only

def yetano_xor(x):
    print('your code here')
    
    #print(" XOR", x, x11, x12, h2, h12, h43, yhat)     # for debugging 
    return yhat

In [39]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 

for i in range(len(x)): 
    yhat = yetano_xor(x[i])
    print(" XOR", x[i], yhat)

 XOR [0 0] [0.02894936]
 XOR [0 1] [0.92881756]
 XOR [1 0] [0.92882857]
 XOR [1 1] [0.0290579]


-------
__Be joyful always!__ 1 Thes.5:16